In [1]:
import numpy as np
import pandas as pd
from simple.renko import Renko
from simple.plotly import chartFigure

In [3]:
T = np.load('../data/tick.npz')['BTCUSDT']
len(T)

889360

In [4]:
rnk = Renko(50, T['PriceA'])
rnk.create_renko()

rnk.bricks[:10]

[{'type': 'first', 'open': 37265.0, 'close': 37265.0},
 {'type': 'down', 'open': 37265.0, 'close': 37215.0},
 {'type': 'down', 'open': 37215.0, 'close': 37165.0},
 {'type': 'up', 'open': 37215.0, 'close': 37265.0, 'low': 37130.0},
 {'type': 'up', 'open': 37265.0, 'close': 37315.0},
 {'type': 'up', 'open': 37315.0, 'close': 37365.0},
 {'type': 'up', 'open': 37365.0, 'close': 37415.0},
 {'type': 'up', 'open': 37415.0, 'close': 37465.0},
 {'type': 'up', 'open': 37465.0, 'close': 37515.0},
 {'type': 'up', 'open': 37515.0, 'close': 37565.0}]

In [5]:
X = pd.DataFrame(rnk.bricks)
X['color'] = np.where(X.type == 'up', 'green', 'red')
X['mid'] = (X.close + X.open) / 2
X.head()

,type,open,close,low,high,color,mid
0,first,37265.0,37265.0,NaN,NaN,red,37265.0
1,down,37265.0,37215.0,NaN,NaN,red,37240.0
2,down,37215.0,37165.0,NaN,NaN,red,37190.0
3,up,37215.0,37265.0,37130.0,NaN,green,37240.0
4,up,37265.0,37315.0,NaN,NaN,green,37290.0


In [6]:
len(X)

1612

In [7]:
import plotly.graph_objects as go
from plotly_resampler import FigureWidgetResampler

In [8]:
fig = FigureWidgetResampler(go.FigureWidget(layout={'height':500, 'template': 'plotly_white',  
                              'margin': dict(l=45, r=5, b=10, t=25, pad=3), 'legend_orientation': 'h'}))

fig.add_scattergl(name='B', mode='markers', y=X.mid, marker=dict(symbol='square', size=5, line_width=1, line_color="black", color=X.color))
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': array(['red', 'red', 'red', ..., 'green', 'red', 're…

In [9]:
chartFigure(Renko=dict(mode='markers', y=X.mid, marker=dict(symbol='square', size=8, line_width=1, line_color="black", marker_color=X.color)))

FigureWidgetResampler({
    'data': [{'mode': 'markers',
              'name': '<b style="color:sandybrown">[R…